# Data Exploration

## Setup

### Downloading Librabies

In [24]:
%pip install pandas
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Importing Librabies

In [25]:
import pandas as pd

## Reading Data

### Read Transactional Data and Non statistical exploration

Reading data and renaming columns as well as dropping uneeded columns

In [26]:
transactional_data = pd.read_csv("../DataSets/TransactionalData/bank.csv")

del transactional_data['CHQ.NO.']
del transactional_data['VALUE DATE']

transactional_data = transactional_data.rename(columns={
    "Account No": "user",
    "DATE": "transaction_date",
    "TRANSACTION DETAILS": "transaction_details",
    " WITHDRAWAL AMT ": "money_in",
    " DEPOSIT AMT ": "money_out",
    "BALANCE AMT": "rolling_balance"
})

Insure all the data is in correct format for exploration

In [27]:
chars_to_remove = [' ', ',']

for char in chars_to_remove:
    transactional_data['money_in'] = transactional_data['money_in'].replace(char, '', regex=True)
    transactional_data['money_out'] = transactional_data['money_out'].replace(char, '', regex=True)
    transactional_data['rolling_balance'] = transactional_data['rolling_balance'].replace(char, '', regex=True)

transactional_data['money_in'] = transactional_data['money_in'].astype(float)
transactional_data['money_out'] = transactional_data['money_out'].astype(float)
transactional_data['rolling_balance'] = transactional_data['rolling_balance'].astype(float)

sorted_data = data = transactional_data.sort_values(by=['user', 'transaction_date'])

rename users and get rid of senstive data

In [28]:
# Get the unique users
unique_users = transactional_data['user'].unique()

# Create a mapping from old usernames to p_1
user_mapping = {old_user: f'p_{i}' for i, old_user in enumerate(unique_users, start=1)}

# Replace the old usernames in the 'users' column with the new usernames
transactional_data['user'] = transactional_data['user'].map(user_mapping)

exploration of users

In [29]:
users = transactional_data.groupby('user').aggregate(
    total_money_in=('money_in', 'sum'),
    total_money_out=('money_out', 'sum'),
    initial_balance=('rolling_balance', 'first'),
    final_balance=('rolling_balance', 'last'),
)

users.sort_values("user")

,total_money_in,total_money_out,initial_balance,final_balance
user,,,,
p_1,1.453974e+08,1.458596e+08,1.000000e+06,4.622000e+05
p_10,1.019351e+11,1.017209e+11,-1.677234e+09,-1.901417e+09
p_2,9.537793e+07,9.565931e+07,9.622000e+05,7.435833e+05
p_3,3.845102e+08,2.703186e+07,7.645833e+05,-3.567348e+08
p_4,4.203179e+08,2.287852e+08,-5.267348e+08,-5.482675e+08
p_5,4.705551e+09,4.705899e+09,-3.780175e+08,-5.479193e+08
p_6,1.006049e+08,1.022096e+08,-5.474193e+08,-5.463146e+08
p_7,1.719608e+10,1.720243e+10,-1.514640e+06,-5.399631e+08
p_8,4.692584e+10,4.587889e+10,-5.399581e+08,-1.586916e+09


### Read User behaviors and Non statistical exploration

#### Typing Data

In [32]:
key_stroke = pd.read_csv("../DataSets/UserBehaviors/Keystroke/free-text.csv")

# Get the unique user IDs
first_10_user_ids = key_stroke['participant'].unique()[:10]

# Filter the data for the first 10 unique users
first_10_users_data = key_stroke[key_stroke['participant'].isin(first_10_user_ids)]

# Drop unneeded columns
first_10_users_data = first_10_users_data.drop(first_10_users_data.columns[-1], axis=1)

# Display head
first_10_users_data

/var/folders/jw/dxpsx41s5xlf3dsmc4b4f4tc0000gn/T/ipykernel_25659/703984424.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  key_stroke = pd.read_csv("../DataSets/UserBehaviors/Keystroke/free-text.csv")


,participant,session,key1,key2,DU.key1.key1,DD.key1.key2,DU.key1.key2,UD.key1.key2,UU.key1.key2
0,p001,1,W,Shift,0.15,-0.796,0.166,-0.946,0.016
1,p001,1,Shift,e,0.962,1.148,1.255,0.186,0.293
2,p001,1,e,Space,0.107,0.172,0.252,0.065,0.145
3,p001,1,Space,b,0.08,0.200,0.280,0.120,0.200
4,p001,1,b,e,0.08,0.320,0.480,0.240,0.400
...,...,...,...,...,...,...,...,...,...
54176,p010,2,s,t,0.127,0.504,0.623,0.377,0.496
54177,p010,2,t,a,0.119,0.119,0.223,0.000,0.104
54178,p010,2,a,r,0.104,0.193,0.264,0.089,0.160
54179,p010,2,r,s,0.071,0.328,0.439,0.257,0.368


#### Mouse movement && Session Info

In [49]:
session_info = pd.read_csv("../DataSets/UserBehaviors/mousedynamics/EVTRACKINFO/EVTRACKINFO.csv", sep='\t')
mouse_movements = pd.read_csv("../DataSets/UserBehaviors/mousedynamics/EVTRACKTRACK/EVTRACKTRACK.csv", sep='\t')

# Drop unneeded columns
del session_info['_id']
del mouse_movements['_id']
del mouse_movements['cursor']


# Filter uneed data
mouse_data = mouse_movements[mouse_movements['event'].str.contains('mouse', case=False, na=False)]

# Display head
mouse_data

,timestamp,xpos,ypos,key,event,user,session_id
0,2020-02-17 08:35:44.485,343.0,65.0,0,mousemove,Alluserspreauth,366758616
1,2020-02-17 08:35:44.785,1010.0,436.0,0,mousemove,Alluserspreauth,366758616
2,2020-02-17 08:35:44.985,894.0,628.0,0,mouseout,Alluserspreauth,366758616
3,2020-02-17 08:35:45.187,884.0,645.0,0,mousemove,Alluserspreauth,366758616
4,2020-02-17 08:35:45.392,879.0,645.0,0,mousemove,Alluserspreauth,366758616
...,...,...,...,...,...,...,...
142686,2020-02-25 09:05:42.182,503.0,287.0,0,mouseout,User1,366981514
142687,2020-02-25 09:05:42.382,446.0,172.0,0,mouseout,User1,366981514
142688,2020-02-25 09:05:42.582,440.0,156.0,0,mousemove,User1,366981514
142689,2020-02-25 09:05:46.228,440.0,156.0,0,mousemove,User1,366981514
